In [2]:
import requests
import bs4
import pandas as pd
import numpy as np
import sys
sys.path.insert(0,'../pygrb/')
from grb.config import DATA_PATH
import pickle
import datetime

# Get data for crossmatching

Get final data matched with GOES

In [2]:
konus_sf_data = pd.read_table('http://www.ioffe.ru/LEA/Solar/KonusWIND_SolarFlares.txt').dropna().drop(columns='#')
konus_sf_data['UT'] = pd.to_datetime(konus_sf_data['TriggerDate']+' '+konus_sf_data['TriggerTime'])
konus_sf_data.to_csv(f'{DATA_PATH}KONUS_SF+GOES_catalog.csv')
konus_sf_data.tail()

,TriggerDate,TriggerTime,TriggerTime.1,Class,GOESTbegin,GOESTmax,GOESTend,UT
1013,2023-01-13,10:12:44.322,36764.322,M3.9,10:05,10:15,10:19,2023-01-13 10:12:44.322
1014,2023-02-11,15:45:01.041,56701.041,X1.1,15:40,15:48,15:54,2023-02-11 15:45:01.041
1015,2023-02-23,06:13:12.583,22392.583,M1.5,06:11,06:14,06:18,2023-02-23 06:13:12.583
1016,2023-02-23,08:09:11.819,29351.819,C7.2,07:58,08:12,08:19,2023-02-23 08:09:11.819
1017,2023-03-03,17:49:23.901,64163.901,X2.1,17:42,17:52,17:59,2023-03-03 17:49:23.901


Load trigger events in waiting mode

In [3]:
def parse_html_table(html_string):
    doc = bs4.BeautifulSoup(html_string)
    table = doc.find('table')
    if table is None:
        return None
    
    data = []
    all_th = table.find_all('th')
    all_heads = [th.get_text() for th in all_th]
    for tr in table.find_all('tr'):
        all_th = tr.find_all('th')
        if all_th:
            continue
        all_td = tr.find_all('td')
        data.append([td.get_text() for td in all_td])
    return pd.DataFrame(data, columns = all_heads)

konus_triggers_data = None
for year in range(2002,2024):
    r = requests.get(f'http://www.ioffe.ru/LEA/kw/wm/{year}/index.html')
    table = parse_html_table(r.text)
    
    konus_triggers_data = pd.concat((konus_triggers_data,table))
konus_triggers_data['Trigger seconds'] = konus_triggers_data['UT'].apply(lambda x: x.split(' ')[-1][1:-1])
konus_triggers_data['UT'] = konus_triggers_data['UT'].apply(lambda x: ' '.join(x.split(' ')[:-1]))
konus_triggers_data['UT'] = pd.to_datetime(konus_triggers_data['UT'])
konus_triggers_data['Dur.'] = konus_triggers_data['Dur.'].astype(int)
konus_triggers_data = konus_triggers_data.dropna().reset_index(drop=True)
konus_triggers_data.to_csv(f'{DATA_PATH}KONUS_waitin_mode_events_catalog.csv')
konus_triggers_data.tail()

,UT,Dur.,Name,Type,Det.,Channels,KW,Others,Comment,Trigger seconds
16887,2019-06-05 23:22:27,3,,,S2,G2,,FER,,84147
16888,2019-06-06 01:55:03,3,GRB 190606A,sGRB,"S1,S2","G1,G2,G3",TRIG,,KW GCN 24784,6903
16889,2019-06-06 09:34:07,18,,,S2,"G1,G2",,,,34447
16890,2019-06-06 13:21:39,38,,,S2,"G1,G2",,,,48099
16891,2019-06-07 01:42:45,35,,,"S1,S2","G1,G2",,FER,,6165


Konus GRBs in waiting mode

In [4]:
konus_triggers_data[konus_triggers_data['Comment'].apply(lambda x:x.find('GRB')>=0)].to_csv(f'{DATA_PATH}KONUS_GRBS.csv')
konus_triggers_data.head()

,UT,Dur.,Name,Type,Det.,Channels,KW,Others,Comment,Trigger seconds
0,2002-01-07 06:44:06,44,,Flare,"S1,S2",G1,,,"SF C3.5, 1.8E-03",24246
1,2002-01-07 14:17:40,65,,Flare,"S1,S2",G1,,,"SF C3.8, 2.9E-03",51460
2,2002-01-08 01:04:13,50,,,S2,G2,,,,3853
3,2002-01-08 17:17:36,197,,Flare,"S1,S2",G1,,,"SF C7.2, 8.2E-03",62256
4,2002-01-08 22:52:41,47,,Flare,"S1,S2",G1,,,"SF C7.5, 6.2E-03",82361


Hurley masterlist

In [5]:
hurley = pd.DataFrame([datetime.datetime.strptime(' '.join(x.split()[1:4]+[x.split()[5]]),'%d %b %y %H:%M:%S') for x in pd.read_excel(f'{DATA_PATH}Hurley catalog.xlsx')['data'].to_list()],columns=['datetime'])

RHESSI and GOES flares

In [6]:
goes_cat = pd.read_csv(f'{DATA_PATH}goes_flares.csv').drop(columns='Unnamed: 0').drop_duplicates('t_max')
goes_cat['t_start'] = pd.to_datetime(goes_cat['t_start'], errors='coerce')
goes_cat['t_max'] = pd.to_datetime(goes_cat['t_max'],errors='coerce')
goes_cat['t_finish'] = pd.to_datetime(goes_cat['t_finish'],errors='coerce')
goes_cat = goes_cat.dropna().reset_index(drop=True)
goes_cat.head()

,id,t_start,t_max,t_finish,type
0,6020,2002-01-01 05:33:00,2002-01-01 05:33:00,2002-01-01 05:38:00,SF
1,6030,2002-01-01 08:59:00,2002-01-01 09:10:00,2002-01-01 09:17:00,C4.5
2,6040,2002-01-01 10:18:00,2002-01-01 10:18:00,2002-01-01 10:18:00,170
3,6050,2002-01-01 12:09:00,2002-01-01 12:09:00,2002-01-01 12:10:00,72
4,6080,2002-01-02 01:15:00,2002-01-02 01:23:00,2002-01-02 01:43:00,SF


In [7]:
with open(f'{DATA_PATH}RHESSI SF catalog.txt') as file:
    RHESSI = [(datetime.datetime.strptime(line.split()[1]+' '+line.split()[2],'%d-%b-%Y %H:%M:%S'),datetime.datetime.strptime(line.split()[1]+' '+line.split()[4],'%d-%b-%Y %H:%M:%S')) for line in file]
RHESSI = pd.DataFrame(RHESSI,columns=['t_start','t_finish'])
RHESSI.head()

,t_start,t_finish
0,2002-02-12 21:29:56,2002-02-12 21:41:48
1,2002-02-12 21:44:08,2002-02-12 21:48:56
2,2002-02-13 00:53:24,2002-02-13 00:57:00
3,2002-02-13 04:22:52,2002-02-13 04:26:56
4,2002-02-13 07:03:52,2002-02-13 07:07:48


# Crossmatching

In [8]:
def find_closest_event(time,list_of_events):
    res = min(list_of_events, key=lambda sub: abs(sub - time))
    index = list(list_of_events).index(res)
    return index

def is_intersected(left_1,right_1,left_2,right_2):
    return (left_1 <= left_2 <= right_1) or \
            (left_1 <= right_2 <= right_1) or \
                (left_2 <= left_1 <= right_2) or \
                (left_2 <= right_1 <= right_2)

In [5]:
with open(f'{DATA_PATH}fixed_durations.pkl','rb') as f:
    final_catalog = pickle.load(f)
df_fixed_durs = pd.DataFrame(final_catalog)
df_fixed_durs = df_fixed_durs[df_fixed_durs['duration']>0]
df_fixed_durs.head()

,datetime,event_time,duration,resolution,acs_flux,chi_2,bins
0,2003-02-14 09:49:43.816,164.804459,194.807866,15.0,1.010940e+05,1.8,"[0.11574814627205356, 0.17832570964751787, 0.1..."
1,2003-02-15 11:13:44.816,131.066354,82.402611,7.5,6.216833e+05,10.7,"[0.0, 0.006219142464290014, 0.0065218629200692..."
2,2003-02-15 15:41:43.816,134.493933,418.676969,30.0,1.410639e+06,4.3,"[0.0, 0.0047844677300591595, 0.008117081742619..."
3,2003-02-16 10:33:43.816,-59.884115,717.356425,60.0,7.009510e+04,0.6,"[0.034826925406951856, 0.002401487333419119, 0..."
4,2003-02-17 02:42:43.816,-149.374733,1372.921857,60.0,7.235530e+05,51.4,"[0.0, 0.0162534253236457, 0.5576596927825326, ..."


In [16]:

df_candidates = pd.read_excel(f'{DATA_PATH}candidates_crossmatched_final (2).xlsx',parse_dates=['datetime'])

candidates = df_candidates.merge(df_fixed_durs,left_on='datetime',right_on='datetime').sort_values('datetime')
mask = 2*candidates['duration_x']<candidates['duration_y']
candidates['resolution'] = np.where(mask,candidates['duration_err'],candidates['resolution'])
candidates['duration'] = np.where(mask,candidates['duration_x'],candidates['duration_y'])
candidates['acs_flux'] = np.where(mask,candidates['integral_flux'],candidates['acs_flux'])
candidates['acs_chi2'] = np.where(mask,candidates['bkg_chi2'],candidates['chi_2'])
candidates['event_t0'] = np.where(mask,candidates['datetime'],candidates.apply(lambda x:x['datetime']+np.timedelta64(int(x['event_time']),'s'),axis=1))


konus_sf_data['left'] = konus_sf_data['right'] = konus_sf_data['UT']
candidates['Konus_SF'] = 0
for i,event in candidates.iterrows():
    idx = find_closest_event(event['datetime'],konus_sf_data['UT'])
    if is_intersected(event['datetime'],
                      event['datetime'] + np.timedelta64(int(event['duration']),'s'),
                      konus_sf_data.iloc[idx]['left'],
                      konus_sf_data.iloc[idx]['right']):
        candidates.loc[candidates.index==i,['Konus_SF']] = 1

Crossmatch with self to remove duplicates

In [36]:
candidates = candidates.sort_values('datetime').reset_index(drop=True)
candidates['left'] = candidates['datetime']
candidates['right'] = candidates.apply(lambda row:row['datetime'] + np.timedelta64(int(row['duration']),'s'),axis=1)

candidates['duplicated'] = 0
pair_index = 1
for i,event in candidates.iterrows():
    if i < candidates.shape[0] - 1:
        idx = i + 1
    else: 
        break
    if is_intersected(event['datetime'],
                      event['datetime'] + np.timedelta64(int(event['duration']),'s'),
                      candidates.iloc[idx]['left'],
                      candidates.iloc[idx]['right']):
        if candidates.iloc[idx]['duplicated'] == 0 and candidates.iloc[i]['duplicated'] == 0:
            pair_index += 1
        candidates.loc[candidates.index==i,['duplicated']] = pair_index
        candidates.loc[candidates.index==idx,['duplicated']] = pair_index

In [38]:
filtered_candidates = candidates[candidates['duplicated'] == 0]
for i,row in candidates[candidates['duplicated'] != 0].iterrows():
    if  (pair := row['duplicated']) not in filtered_candidates['duplicated'].unique():
        filtered_candidates = pd.concat((filtered_candidates,candidates[candidates['duplicated']==pair].sort_values('duration').head(1)))
filtered_candidates = filtered_candidates.sort_values('datetime').drop(columns=['duplicated'])

Crossmatch with grbs

In [39]:
konus_triggers_data = pd.read_csv(f'{DATA_PATH}KONUS_GRBS.csv',parse_dates=['UT'])
konus_triggers_data['left'] = konus_triggers_data['UT']
konus_triggers_data['right'] = konus_triggers_data.apply(lambda row:row['UT'] + np.timedelta64(row['Dur.'],'s'),axis=1)

filtered_candidates['grb'] = 0
filtered_candidates['comment'] = ''
for i,event in filtered_candidates.iterrows():
    idx = find_closest_event(event['datetime'],konus_triggers_data['UT'])
    if is_intersected(event['datetime'],
                      event['datetime'] + np.timedelta64(int(event['duration']),'s'),
                      konus_triggers_data.iloc[idx]['left'],
                      konus_triggers_data.iloc[idx]['right']):
        filtered_candidates.loc[filtered_candidates.index==i,['grb']] = 1
        filtered_candidates.loc[filtered_candidates.index==i,['comment']] = konus_triggers_data.iloc[idx]['Comment']

In [42]:
hurley['left'] = hurley['right'] = hurley['datetime']

filtered_candidates['hurley'] = 0
for i,event in filtered_candidates.iterrows():
    idx = find_closest_event(event['datetime'],hurley['datetime'])
    if is_intersected(event['datetime'],
                      event['datetime'] + np.timedelta64(int(event['duration']),'s'),
                      hurley.iloc[idx]['left'],
                      hurley.iloc[idx]['right']):
        filtered_candidates.loc[filtered_candidates.index==i,['hurley']] = 1

crossmatch with solar flares

In [43]:
filtered_candidates['goes'] = 0
for i,event in filtered_candidates.iterrows():
    idx = find_closest_event(event['datetime'],goes_cat['t_start'])
    if is_intersected(event['datetime'],
                      event['datetime'] + np.timedelta64(int(event['duration']),'s'),
                      goes_cat.iloc[idx]['t_start'],
                      goes_cat.iloc[idx]['t_finish']):
        filtered_candidates.loc[filtered_candidates.index==i,['goes']] = 1

filtered_candidates['rhessi'] = 0
for i,event in filtered_candidates.iterrows():
    idx = find_closest_event(event['datetime'],RHESSI['t_start'])
    if is_intersected(event['datetime'],
                      event['datetime'] + np.timedelta64(int(event['duration']),'s'),
                      RHESSI.iloc[idx]['t_start'],
                      RHESSI.iloc[idx]['t_finish']):
        filtered_candidates.loc[filtered_candidates.index==i,['rhessi']] = 1

# Tag remaining events

Apply background criteria

In [47]:
filtered_candidates['bkg'] = filtered_candidates.apply(lambda row:row['chi_2']>3 if row['duration']*3 <= 3000 else row['chi_2']>4.5,axis=1).astype(int)

Rest are candidates

In [48]:
filtered_candidates['candidates'] = filtered_candidates.apply(lambda row: (row['Konus_SF'] + row['grb'] + row['hurley'] + row['goes'] + row['rhessi'] + row['bkg']) == 0, axis = 1).astype(int)

In [50]:
filtered_candidates.columns

Index(['Unnamed: 0', 'datetime', 'distance', 'duration_x', 'duration_err',
       'integral_flux', 'integral_flux_err', 'significance', 'bkg_chi2', 'Bkg',
       'Hurley', 'RHESSI', 'Konus', 'GOES', 'Candidate', 'event_time',
       'duration_y', 'resolution', 'acs_flux', 'chi_2', 'bins', 'duration',
       'acs_chi2', 'event_t0', 'Konus_SF', 'left', 'right', 'grb', 'comment',
       'hurley', 'goes', 'rhessi', 'bkg', 'candidates'],
      dtype='object')

In [52]:
filtered_candidates = filtered_candidates[['datetime','duration','resolution','event_t0','acs_flux', 'chi_2','Konus_SF','grb', 'comment',
       'hurley', 'goes', 'rhessi', 'bkg', 'candidates','bins']]

In [55]:
filtered_candidates = filtered_candidates[filtered_candidates['acs_flux']>0].reset_index(drop=True)
filtered_candidates.head()

,datetime,duration,resolution,distance,event_t0,acs_flux,chi_2,Konus_SF,grb,comment,hurley,goes,rhessi,bkg,candidates
0,2003-02-12 03:38:53.978,21.875011,3.125,0.55,2003-02-12 04:04:53.978,5.042429e+03,3.4,0,0,,0,0,0,1,0
1,2003-02-14 04:04:43.816,540.005252,20.000,0.93,2003-02-14 04:06:42.816,2.638550e+05,1.7,0,0,,0,1,1,0,0
2,2003-02-14 09:49:43.816,194.807866,15.000,1.17,2003-02-14 09:52:27.816,1.010940e+05,1.8,0,0,,0,0,0,0,1
3,2003-02-15 11:14:03.816,80.001214,5.000,0.23,2003-02-15 11:15:55.816,6.221181e+05,8.5,0,0,,0,0,0,1,0
4,2003-02-15 15:43:43.816,360.003661,20.000,0.04,2003-02-15 15:45:03.816,1.400081e+06,3.0,0,0,,0,0,1,0,0


In [10]:
filtered_candidates.to_pickle(f'{DATA_PATH}filtered_candidates.pkl')

In [9]:
filtered_candidates = pd.read_pickle(f'{DATA_PATH}filtered_candidates.pkl')